In [1]:
import pandas as pd
import numpy as np
from scipy.signal import welch
from scipy.stats import linregress

# Step 1: Load EEG data from CSV file
df = pd.read_csv('downsampled_final_125Hz.csv', usecols=lambda column: column not in ['Unnamed: 0.1', 'Unnamed: 0'])

# Define EEG channel columns (assuming 'channel_1' to 'channel_8' are the relevant EEG channels)
eeg_channels = [f'channel_{i}' for i in range(1, 9)]

# Define sampling rate (Hz)
fs = 125  # Assuming 125Hz as the sampling rate

# Define window size (e.g., 500ms) and step size (e.g., 100ms)
window_size = int(0.5 * fs)  # 500ms window, which equals 62 samples at 125Hz
step_size = int(0.1 * fs)    # 100ms step, equals 12 samples

# Step 2: Define a function to extract features from each window
def extract_features_with_sliding_window(df, eeg_channels, fs=125, window_size=62, step_size=12):
    feature_list = []

    # Sliding window over the entire dataset
    for i in range(0, len(df) - window_size + 1, step_size):
        window_df = df.iloc[i:i + window_size]  # Extract the current window of data
        features = {}

        # Extract participant ID and timestep (from the first row of the window)
        features['participant_id'] = window_df['participant_id'].iloc[0]
        features['timestep_start'] = window_df['timestep'].iloc[0]
        features['timestep_end'] = window_df['timestep'].iloc[-1]

        # 1. Time-Domain Features (Max, Min, Mean, Amplitude Range)
        for channel in eeg_channels:
            channel_values = window_df[channel].values
            features[f'{channel}_max'] = np.max(channel_values)
            features[f'{channel}_min'] = np.min(channel_values)
            features[f'{channel}_mean'] = np.mean(channel_values)
            features[f'{channel}_amplitude_range'] = np.max(channel_values) - np.min(channel_values)

        # 2. Frequency-Domain Features (Alpha, Beta, Gamma Power using Welch method)
        for channel in eeg_channels:
            nperseg = min(len(window_df[channel].values), fs)
            freqs, psd = welch(window_df[channel].values, fs=fs, nperseg=nperseg)
            alpha_power = np.sum(psd[(freqs >= 8) & (freqs <= 12)])
            beta_power = np.sum(psd[(freqs >= 13) & (freqs <= 30)])
            gamma_power = np.sum(psd[(freqs >= 30) & (freqs <= 50)])
            
            features[f'{channel}_alpha_power'] = alpha_power
            features[f'{channel}_beta_power'] = beta_power
            features[f'{channel}_gamma_power'] = gamma_power
        
        # 3. Fast Fourier Transform (FFT) Features
        for channel in eeg_channels:
            channel_values = window_df[channel].values
            fft_values = np.fft.fft(channel_values)
            fft_freqs = np.fft.fftfreq(len(fft_values), d=1/fs)

            # Consider only the positive frequencies
            positive_fft_values = fft_values[:len(fft_values) // 2]
            positive_fft_freqs = fft_freqs[:len(fft_freqs) // 2]

            # FFT features
            total_power = np.sum(np.abs(positive_fft_values) ** 2)  # Total power in frequency domain
            dominant_freq = positive_fft_freqs[np.argmax(np.abs(positive_fft_values))]  # Frequency with the maximum amplitude

            features[f'{channel}_fft_total_power'] = total_power
            features[f'{channel}_fft_dominant_frequency'] = dominant_freq

        # 4. Signal Slope (Dynamic Feature)
        for channel in eeg_channels:
            channel_values = window_df[channel].values
            slope, intercept, r_value, p_value, std_err = linregress(np.arange(len(channel_values)), channel_values)
            features[f'{channel}_slope'] = slope

        # 5. Inter-Channel Differences (Spatial Features)
        for i in range(len(eeg_channels)):
            for j in range(i + 1, len(eeg_channels)):
                channel_i = window_df[eeg_channels[i]].values
                channel_j = window_df[eeg_channels[j]].values
                features[f'{eeg_channels[i]}_{eeg_channels[j]}_diff'] = np.mean(channel_i - channel_j)

        # Add the trigger value (assuming we're taking the trigger from the last timestep in the window)
        features['trigger'] = window_df['trigger'].iloc[-1]

        # Append extracted features for this window
        feature_list.append(features)

    # Convert list of feature dicts into a DataFrame
    return pd.DataFrame(feature_list)

# Step 3: Apply feature extraction with sliding window
features_df = extract_features_with_sliding_window(df, eeg_channels, fs=fs, window_size=window_size, step_size=step_size)

# Step 4: Save extracted features to a new CSV file
features_df.to_csv('extracted_features.csv', index=False)

# Step 5: Optional - print first few rows of the resulting features DataFrame to verify
print(features_df.head())

   participant_id  timestep_start  timestep_end  channel_1_max  channel_1_min  \
0               1    15887.065728   1339.265314      20.313471     -14.123051   
1               1     1714.728696   1375.508210      20.313471     -14.123051   
2               1     1546.212343   1399.907876      16.657329     -14.123051   
3               1     1500.411118   1430.686234       8.870569     -14.123051   
4               1     1491.993061   1456.833291       8.870569     -14.123051   

   channel_1_mean  channel_1_amplitude_range  channel_2_max  channel_2_min  \
0       -1.045016                  34.436521      13.381414     -19.338251   
1       -0.738558                  34.436521      13.381414     -14.061762   
2       -4.120847                  30.780380      12.531670     -14.061762   
3       -4.422534                  22.993620       6.202993     -14.061762   
4       -3.978650                  22.993620      11.550842     -13.917930   

   channel_2_mean  ...  channel_4_channel_6_

| **Feature Name**                        | **Description**                                                                             |
|-----------------------------------------|---------------------------------------------------------------------------------------------|
| `participant_id`                        | Unique identifier for the participant, used to differentiate signal data from different individuals. |
| `timestep_start`                        | The starting timestep of the sliding window, marking the beginning of the data segment.      |
| `timestep_end`                          | The ending timestep of the sliding window, marking the end of the data segment.              |
| `channel_X_max`                         | Maximum value of channel `X` within the current window, capturing the peak signal.          |
| `channel_X_min`                         | Minimum value of channel `X` within the current window, capturing the lowest signal.        |
| `channel_X_mean`                        | Mean value of channel `X` within the current window, representing the overall signal level. |
| `channel_X_amplitude_range`             | Difference between the max and min values of channel `X`, showing the signal's amplitude range. |
| `channel_X_alpha_power`                 | Power of the alpha band (8-12 Hz) for channel `X`, linked to relaxation and attention.      |
| `channel_X_beta_power`                  | Power of the beta band (13-30 Hz) for channel `X`, associated with cognitive activity and focus. |
| `channel_X_gamma_power`                 | Power of the gamma band (30-50 Hz) for channel `X`, related to high-level cognitive processes. |
| `channel_X_fft_total_power`             | Total power in the frequency domain of channel `X`, representing the signal's overall energy. |
| `channel_X_fft_dominant_frequency`      | Dominant frequency of channel `X`, indicating the most significant frequency component in the signal. |
| `channel_X_slope`                       | Slope of the signal for channel `X` in the current window, indicating the trend (rising or falling) of the signal. |
| `channel_X_channel_Y_diff`              | Average difference between signals of channel `X` and channel `Y`, showing inter-channel relationships. |
| `trigger`                               | Event label or trigger value, indicating whether a specific event (e.g., P300) occurred within the window. |
